In [36]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
import fine_tune.annotation_utils as a_utils
import fine_tune.llm_utils as llm_utils
import fine_tune.message_utils as m_utils
from fine_tune.env import (
    BRAT_DATA_PATH,
)
from fine_tune import utils
import split_training_data as std
from job_preset import JOB_PRESET
import os

load_dotenv()


True

## Load training job

### Load from preset

In [39]:
job_desc = 'data_class-sent_data-v2-d4'

job_preset = JOB_PRESET[job_desc]

data_entities = job_preset.load_data()
all_data = job_preset.as_training_data(data_entities)
training_set_indices, validation_set_indices = job_preset.training_data_splitter(data_entities)

# base_model = 'gpt-4o-mini-2024-07-18'
base_model = 'gpt-4o-2024-08-06'

suffix = job_desc

job_desc, len(data_entities), len(training_set_indices), len(validation_set_indices), base_model, all_data[10]

('data_class-sent_data-v2-d4',
 1067,
 120,
 30,
 'gpt-4o-2024-08-06',
 {'messages': [{'role': 'system',
    'content': 'You are an annotation expert, and your task is to map phrases concerning user private data to one of the 211 categories below.\nYou will be given a list of categories/terms as a hierarchy, meaning that some categories will have subcategories.\nYou will also be given definitions of each of these categories. Please rely on these definitions.\nYou always have to choose the category that matches the given phrase the closest and most precise.\nThe categories that are "deeper" in the hierarchy are more precise, so you should use them whenever they apply, and only use the parent category, if none of the child category matches the phrase.\nHere is the hierarchy of terms:\n\nInternal\n\tAuthenticating\n\t\tPassword\n\t\tPINCode\n\t\tSecretText\n\tKnowledgeBelief\n\t\tPhilosophicalBelief\n\t\tReligiousBelief\n\t\tThought\n\tPreference\n\t\tFavorite\n\t\t\tFavoriteColor\n\t\t\t

### Or, custom load

#### Load brat dataset

In [2]:

import dataclasses

from pybrat.parser import BratParser, Entity, Event, Example, Relation

# Initialize a parser.
brat = BratParser(error="ignore")
# brat = BratParser()
annotations = brat.parse(BRAT_DATA_PATH)


In [3]:
# data_entities = a_utils.get_data_entities_of_segments(annotations)
data_entities = a_utils.get_data_entities_of_sentences(annotations)

len(data_entities), data_entities[10]

(1067,
 {'segment': 'INTRODUCTION\n\nDomains Holdings Group Limited (the "Company"), offers a global network of content-rich web, mobile and social sites, including user acquisition and media buying (including paid search, display, mobile and social advertising), content optimization (including key word search, off-page and on-page optimization), and affiliate program, both in the web and mobile ecosystem (including through websites or mobile apps operated by Company; collectively, the "Services").\nThe Company is deeply committed to safeguard the privacy expectations of its end users ("User(s)", "you" or "your"). Accordingly, we have put in place this Privacy Policy, which outlines our data protection practices, including how we collect, use, disclose and protect your Personal Information, as well as your rights with respect to your Personal Information.\nIn this Privacy Policy, you can learn about:\nWHAT INFORMATION WE COLLECT?\nMINORS\nHOW WE USE THE COLLECTED INFORMATION?\nMARKETIN

#### Select training data

In [4]:
# training_set_indices, validation_set_indices = std.simple_split(data_entities)
training_set_indices, validation_set_indices = std.better_split(data_entities)

len(training_set_indices), len(validation_set_indices)

(40, 8)

#### Set parameters

In [92]:
# all_data = m_utils.as_training_data_for_data_span_of_segment(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_segment_1_1(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence(data_entities)
# all_data = m_utils.as_training_data_for_data_span_of_sentence_1(data_entities)  # Haven't used here
all_data = m_utils.as_training_data_for_data_span_of_sentence_only(data_entities)
# all_data = m_utils.as_training_data_for_data_classification_of_segment(data_entities)
# all_data = m_utils.as_training_data_for_data_classification_of_segment_gradual(data_entities)

# base_model = 'gpt-4o-2024-08-06'
base_model = 'gpt-4o-mini-2024-07-18'
suffix = 'data_entity-sent_data-ver2'
# suffix = 'sent-data-entities-30-train-5-val-with-empty'

job_desc = suffix

all_data[0]

{'messages': [{'role': 'system',
   'content': 'You are an annotation expert. You will be given a segment of a privacy policy of a web or mobile application, and will be asked to annotate entities in it.\nPlease annotate the given sentence of privacy policy for data entities, and adhere to the following guidelines:\n\nIMPORTANT: Filtering Out General Phrases\nBefore annotating, carefully check each potential data entity. DO NOT annotate general phrases that do not provide specific data types.\nExamples of general phrases to omit include, but are not limited to:\n\n"the information we collect about you"\n"other data"\n"any information"\n\nIf a phrase does not clearly indicate a specific type of personal data, DO NOT include it in your annotations.\n\nData entities are refers to the phrases that mention PERSONAL DATA OF THE USER which is being mentioned in one of the following context types:\n1. first-party-collection-use - the policy segment mentions collection, usage, or processing of 

## Fine-tune model with specified training data

In [40]:
job_desc_dir, fine_tune_job, test_set = llm_utils.fine_tune_with_data(
    all_data, training_set_indices, validation_set_indices,
    basemodel=base_model,
    fine_tune_args={'suffix':suffix},
    desc=job_desc)

job_desc_dir, fine_tune_job, len(test_set), test_set[0]

Constructing training and validation data files (locally)...
Uploading training and validation data files...
Data files uploading (API calls returned).
Creating fine-tuning job...
Created fine-tuning job.
Job description saved.


(PosixPath('../out/fine_tune-2024-10-04-22-33-38-gpt-4o-2024-08-06'),
 FineTuningJob(id='ftjob-PZ6US9G9qq3uLazHkIJpnN6z', created_at=1728077622, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=[], seed=159295259, status='validating_files', trained_tokens=None, training_file='file-68M2b6S0rJ1egI0k5fUjZ9mF', validation_file='file-mf9Qu6qJejINq09SLhqLOSlW', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='renyuneyun-university-of-oxford', entity=None, name=None, tags=None, run_id='ftjob-PZ6US9G9qq3uLazHkIJpnN6z'))], user_provided_suffix='data_class-sent_data-v2-d4'),
 917,
 {'messages': [{'role': 'system',
    'content': 'You are an annotation expert, a

In [9]:
%debug

> /home/ryey/workspaces/oxford/PP-DToU/fine-tune/src/llm_utils.py(50)fine_tune_with_data()
     48     fl.symlink_to(job_desc_dir)
     49 
---> 50     training_set = [all_data[i] for i in training_set_indices]
     51     validation_set = [all_data[i] for i in validation_set_indices]
     52     test_set = [all_data[i] for i in set(range(len(all_data))) - set(training_set_indices) - set(validation_set_indices)]

237


In [41]:
llm_utils.await_fine_tune_finish_and_clean_up(wait_for_job_completion=False)

FineTuningJob(id='ftjob-61c0FVgYkOfjZaMwRpOjNOzo', created_at=1726265855, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:rui:30-train-5-val-with-empty-from-api:A790JXld', finished_at=1726266269, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-B2C2pNzAq4paAOvhIYdFJlSv', result_files=['file-1TTTIJ6t7DVXg51yYafkwd4J'], seed=1504396448, status='succeeded', trained_tokens=68019, training_file='file-iYtsamFpHaR3FyqGhHWNeMVm', validation_file='file-C1S6NC5e9VWZrwgmuFzkDxVR', estimated_finish=None, integrations=[], user_provided_suffix='30_train-5_val-with_empty-from_api')

In [41]:
llm_utils.clear_server_data()